In [ ]:
%cd "../"

In [2]:
import onnxruntime as ort

import torch
import numpy as np

import matplotlib.pyplot as plt
from scipy.ndimage import zoom

### UNet++

#### ONNX Runtime

In [3]:
# Input and preprocessing
image = np.load('samples/0001_0170.npy')
img_size = 128
x, y = image.shape
if x != img_size and y != img_size:
    inputs = zoom(image, (img_size / x, img_size / y), order=0)
    
inputs = inputs[np.newaxis, np.newaxis, :, :]

# Initialize model
provider = ['CUDAExecutionProvider', 'CPUExecutionProvider']
ort_session = ort.InferenceSession("OnnxModels/onnx_model2.onnx", providers=provider)

# Warmup step
inp = {ort_session.get_inputs()[0].name: inputs}
out = ort_session.run(None, inp)
out = np.argmax(out[0], axis=1)

In [4]:
%%timeit
inp = {ort_session.get_inputs()[0].name: inputs}
out = ort_session.run(None, inp)

5.27 ms ± 36.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Pytorch runtime

In [5]:
# Input and preprocessing
image = np.load('samples/0001_0170.npy')
img_size = 128
x, y = image.shape
if x != img_size and y != img_size:
    inputs = zoom(image, (img_size / x, img_size / y), order=0)
    
inputs = torch.tensor(inputs).unsqueeze(0).unsqueeze(0).cuda()

# Intialize model
model = torch.load("TorchModels/model1.pth").cuda()
model.eval()
model = model.cuda()
model = torch.compile(model, backend="eager")

# Warmup step
with torch.inference_mode():
    output = model(inputs)

In [6]:
%%timeit
with torch.inference_mode():
    output = model(inputs)

5.82 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## RotCAtt-TransUNet-plusplus

### ONNX Runtime

In [3]:
# Input and preprocessing
img_size = 128
vol = []
for index in range(167, 174, 1):
    slice = np.load(f'samples/0001_{index:04d}.npy')
    x, y = slice.shape
    if x != img_size and y != img_size:
        slice = zoom(slice, (img_size / x, img_size / y), order=0)
    vol.append(slice)

inputs = np.array(vol)[:, np.newaxis, :, :]

# Initialize model
provider = ['CUDAExecutionProvider', 'CPUExecutionProvider']
ort_session = ort.InferenceSession("OnnxModels/onnx_model4.onnx", providers=provider)

# Warmup step
inp = {ort_session.get_inputs()[0].name: inputs}
out = ort_session.run(None, inp)

In [4]:
%%timeit
out = ort_session.run(None, inp)

37.7 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Pytorch runtime

### GPU

In [5]:
# Input and preprocessing
img_size = 128
vol = []
for index in range(167, 174, 1):
    slice = np.load(f'samples/0001_{index:04d}.npy')
    x, y = slice.shape
    if x != img_size and y != img_size:
        slice = zoom(slice, (img_size / x, img_size / y), order=0)
    vol.append(slice)

vol = np.array(vol)
inputs = torch.tensor(vol).unsqueeze(1).cuda()


# Intialize model
model = torch.load("TorchModels/model2.pth").cuda()
model.eval()
model = model.cuda()
model = torch.compile(model, backend="eager")

# Warmup step
with torch.inference_mode():
    output = model(inputs)

In [6]:
%%timeit
with torch.inference_mode():
    output = model(inputs)

86.5 ms ± 5.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### CPU

In [5]:
# Input and preprocessing
img_size = 128
vol = []
for index in range(167, 174, 1):
    slice = np.load(f'samples/0001_{index:04d}.npy')
    x, y = slice.shape
    if x != img_size and y != img_size:
        slice = zoom(slice, (img_size / x, img_size / y), order=0)
    vol.append(slice)

vol = np.array(vol)
inputs = torch.tensor(vol).unsqueeze(1)


# Intialize model
model = torch.load("TorchModels/model2.pth").cuda()
model.eval()
model = model.cpu()
model = torch.compile(model, backend="eager")

# Warmup step
with torch.inference_mode():
    output = model(inputs)

In [6]:
%%timeit
with torch.inference_mode():
    output = model(inputs)

621 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
